In [1]:
import pandas as pd 
import numpy as np 
import glob
import matplotlib.pyplot as plt
from time import time

from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, recall_score
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold

# Differnt types of Naive Bayes Classifiers
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

pd.set_option('display.max_columns', None)

In [13]:
df = pd.read_csv('../twitter_data_train_multiclass.csv')
test = pd.read_csv('../twitter_data_test_multiclass.csv')

In [14]:
df.head()

,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,default_profile,default_profile_image,geo_enabled,profile_image_url,profile_use_background_image,profile_background_tile,protected,verified,description,account_type,tweets_list,tweet_frequency,number_of_tags,number_of_mentions,tweet_weekend_frequency,tweet_weekday_frequency,following_to_followers_ratio,username_length,screen_name_length,username_spec_char_count,screen_name_spec_char_count,has_url,tweets_list_processed,description_processed,tweets_0,tweets_1,tweets_2,tweets_3,tweets_4,tweets_5,tweets_6,tweets_7,tweets_8,tweets_9,tweets_10,tweets_11,tweets_12,tweets_13,tweets_14,tweets_15,tweets_16,tweets_17,tweets_18,tweets_19,tweets_20,tweets_21,tweets_22,tweets_23,tweets_24,tweets_25,tweets_26,tweets_27,tweets_28,tweets_29,tweets_30,tweets_31,tweets_32,tweets_33,tweets_34,tweets_35,tweets_36,tweets_37,tweets_38,tweets_39,tweets_40,tweets_41,tweets_42,tweets_43,tweets_44,tweets_45,tweets_46,tweets_47,tweets_48,tweets_49,tweets_50,tweets_51,tweets_52,tweets_53,tweets_54,tweets_55,tweets_56,tweets_57,tweets_58,tweets_59,tweets_60,tweets_61,tweets_62,tweets_63,tweets_64,tweets_65,tweets_66,tweets_67,tweets_68,tweets_69,tweets_70,tweets_71,tweets_72,tweets_73,tweets_74,tweets_75,tweets_76,tweets_77,tweets_78,tweets_79,tweets_80,tweets_81,tweets_82,tweets_83,tweets_84,tweets_85,tweets_86,tweets_87,tweets_88,tweets_89,tweets_90,tweets_91,tweets_92,tweets_93,tweets_94,tweets_95,tweets_96,tweets_97,tweets_98,tweets_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,account_type_multi
0,0,82118682,davide gazzÃ¨,davidegazze,88,19,39,9,0,NaN,0.0,0.0,0.0,http://a0.twimg.com/profile_images/1700682774/...,1.0,1.0,NaN,NaN,NaN,1,['#ballarÃ\x83Â² quindi #monti non Ã\x83Â¨ sta...,1.491525,40.0,38.0,1.142857,1.600000,2.052632,13,11,2,0,0,ballarx83 quindi monti non x83 stato un buon t...,NaN,0.091977,0.034554,0.000000,0.059238,0.000000,0.000000,0.000000,0.056452,0.048987,0.058207,0.11607,0.125471,0.0,0.000000,0.151018,0.000000,0.000000,0.058498,0.15312,0.118353,0.439767,0.000000,0.200249,0.0,0.000000,0.178702,0.030877,0.000000,0.000000,0.000000,0.000000,0.0,0.030396,0.0,0.0,0.386811,0.028850,0.0,0.111074,0.000000,0.069019,0.071754,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.031558,0.000000,0.000000,0.000000,0.0,0.000000,0.063091,0.000000,0.000000,0.161387,0.059978,0.000000,0.294536,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.083316,0.000000,0.0,0.000000,0.136500,0.0,0.000000,0.114215,0.3741

In [16]:
df.columns

Index(['Unnamed: 0', 'id', 'name', 'screen_name', 'statuses_count',
       'followers_count', 'friends_count', 'favourites_count', 'listed_count',
       'url',
       ...
       'description_92', 'description_93', 'description_94', 'description_95',
       'description_96', 'description_97', 'description_98', 'description_99',
       'has_face', 'account_type_multi'],
      dtype='object', length=236)

## Get to the correct data format

In [17]:
to_drop = ['Unnamed: 0', 'screen_name', 'url', 'profile_image_url', 'description',
           'id', 'name', 'account_type', 'tweets_list', 'tweets_list_processed',
          'description_processed', 'protected', 'verified', 'account_type_multi',
          'profile_background_tile']

X_train, y_train = df.drop(to_drop, axis=1), df['account_type_multi']

In [18]:
X_test, y_test = test.drop(to_drop, axis=1), test['account_type_multi']

In [19]:
X_train.head()


,statuses_count,followers_count,friends_count,favourites_count,listed_count,default_profile,default_profile_image,geo_enabled,profile_use_background_image,profile_background_tile,tweet_frequency,number_of_tags,number_of_mentions,tweet_weekend_frequency,tweet_weekday_frequency,following_to_followers_ratio,username_length,screen_name_length,username_spec_char_count,screen_name_spec_char_count,has_url,tweets_0,tweets_1,tweets_2,tweets_3,tweets_4,tweets_5,tweets_6,tweets_7,tweets_8,tweets_9,tweets_10,tweets_11,tweets_12,tweets_13,tweets_14,tweets_15,tweets_16,tweets_17,tweets_18,tweets_19,tweets_20,tweets_21,tweets_22,tweets_23,tweets_24,tweets_25,tweets_26,tweets_27,tweets_28,tweets_29,tweets_30,tweets_31,tweets_32,tweets_33,tweets_34,tweets_35,tweets_36,tweets_37,tweets_38,tweets_39,tweets_40,tweets_41,tweets_42,tweets_43,tweets_44,tweets_45,tweets_46,tweets_47,tweets_48,tweets_49,tweets_50,tweets_51,tweets_52,tweets_53,tweets_54,tweets_55,tweets_56,tweets_57,tweets_58,tweets_59,tweets_60,tweets_61,tweets_62,tweets_63,tweets_64,tweets_65,tweets_66,tweets_67,tweets_68,tweets_69,tweets_70,tweets_71,tweets_72,tweets_73,tweets_74,tweets_75,tweets_76,tweets_77,tweets_78,tweets_79,tweets_80,tweets_81,tweets_82,tweets_83,tweets_84,tweets_85,tweets_86,tweets_87,tweets_88,tweets_89,tweets_90,tweets_91,tweets_92,tweets_93,tweets_94,tweets_95,tweets_96,tweets_97,tweets_98,tweets_99,description_0,description_1,description_2,description_3,description_4,description_5,description_6,description_7,description_8,description_9,description_10,description_11,description_12,description_13,description_14,description_15,description_16,description_17,description_18,description_19,description_20,description_21,description_22,description_23,description_24,description_25,description_26,description_27,description_28,description_29,description_30,description_31,description_32,description_33,description_34,description_35,description_36,description_37,description_38,description_39,description_40,description_41,description_42,description_43,description_44,description_45,description_46,description_47,description_48,description_49,description_50,description_51,description_52,description_53,description_54,description_55,description_56,description_57,description_58,description_59,description_60,description_61,description_62,description_63,description_64,description_65,description_66,description_67,description_68,description_69,description_70,description_71,description_72,description_73,description_74,description_75,description_76,description_77,description_78,description_79,description_80,description_81,description_82,description_83,description_84,description_85,description_86,description_87,description_88,description_89,description_90,description_91,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face
0,88,19,39,9,0,0.0,0.0,0.0,1.0,1.0,1.491525,40.0,38.0,1.142857,1.600000,2.052632,13,11,2,0,0,0.091977,0.034554,0.000000,0.059238,0.000000,0.000000,0.000000,0.056452,0.048987,0.058207,0.11607,0.125471,0.0,0.000000,0.151018,0.000000,0.000000,0.058498,0.15312,0.118353,0.439767,0.000000,0.200249,0.0,0.000000,0.178702,0.030877,0.000000,0.000000,0.000000,0.000000,0.0,0.030396,0.0,0.0,0.386811,0.028850,0.0,0.111074,0.000000,0.069019,0.071754,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.031558,0.000000,0.000000,0.000000,0.0,0.000000,0.063091,0.000000,0.000000,0.161387,0.059978,0.000000,0.294536,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.083316,0.000000,0.0,0.000000,0.136500,0.0,0.000000,0.114215,0.374189,0.000000,0.0,0.000000,0.000000,0.0,0.029388,0.000000,0.177164,0.180592,0.000000,0.267742,0.0,0.091636,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

## Mixed NB Approach 1 
train a gaussian NB model on all features

In [20]:
model_G = GaussianNB()
clf_G = model_G.fit(X_train, y_train)
y_pred = model_G.predict(X_test)

In [21]:
# train classification report
y_train_pred = model_G.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.87      0.46      0.60      2338
           1       0.47      0.98      0.63      2706
           2       0.25      0.54      0.34       302
           3       0.85      0.30      0.45      3706
           4       0.53      0.69      0.60       314
           5       0.45      1.00      0.62        80

    accuracy                           0.56      9446
   macro avg       0.57      0.66      0.54      9446
weighted avg       0.71      0.56      0.54      9446



In [22]:
# test classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.45      0.59       413
           1       0.47      0.99      0.63       460
           2       0.24      0.55      0.34        58
           3       0.88      0.32      0.47       666
           4       0.51      0.66      0.57        53
           5       0.50      0.94      0.65        17

    accuracy                           0.56      1667
   macro avg       0.58      0.65      0.54      1667
weighted avg       0.72      0.56      0.54      1667



## Mixed NB Approach 2
train three separate models using continuous, binary and categorical variables and then train the final model based on predictions from the first three models

In [23]:
binary_cols = ['default_profile', 'default_profile_image', 'geo_enabled', 'default_profile_image', 'profile_use_background_image',                          'profile_background_tile', 'has_url', 'has_face']
continuous_cols = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 'listed_count', 'tweet_frequency',    
                    'number_of_tags', 'number_of_mentions', 'tweet_weekend_frequency', 'tweet_weekday_frequency',               
                    'following_to_followers_ratio', 'username_length', 'screen_name_length', 'username_spec_char_count', 
                    'screen_name_spec_char_count']												

In [24]:
train_binary = X_train[binary_cols]
train_continuous = X_train[continuous_cols]
train_text = X_train.drop(binary_cols+continuous_cols, axis=1) 

In [25]:
test_binary = X_test[binary_cols]
test_continuous = X_test[continuous_cols]
test_text = X_test.drop(binary_cols+continuous_cols, axis=1) 

In [26]:
# ----- Fit three models -----

model_G = GaussianNB()
clf_G = model_G.fit(train_continuous, y_train)

model_M = MultinomialNB()
clf_M = model_M.fit(train_text, y_train)

model_B = BernoulliNB()
clf_B = model_B.fit(train_binary, y_train)

# ----- Get probability predictions from each model -----
# On training data
G_train_probas = model_G.predict_proba(train_continuous)
M_train_probas = model_M.predict_proba(train_text)
B_train_probas = model_B.predict_proba(train_binary)

# And on testing data
G_test_probas = model_G.predict_proba(test_continuous)
M_test_probas = model_M.predict_proba(test_text)
B_test_probas = model_B.predict_proba(test_binary)

# Combine probability prediction for class=1 from both models
X_new_train = np.c_[(G_train_probas[:,1], M_train_probas[:,1], B_train_probas[:,1])] # Train
X_new_test = np.c_[(G_test_probas[:,1], M_test_probas[:,1], B_test_probas[:,1])] # Test


# ----- Fit Gaussian model on the X_new -----
model = GaussianNB()
clf = model.fit(X_new_train, y_train)
y_pred = model.predict(X_new_test)

In [27]:
# train classification report
y_train_pred = model.predict(X_new_train)
print(classification_report(y_train, y_train_pred, digits=5))

              precision    recall  f1-score   support

           0    0.77404   0.28571   0.41737      2338
           1    0.97411   0.91759   0.94500      2706
           2    0.31469   0.29801   0.30612       302
           3    0.76551   0.63249   0.69267      3706
           4    0.25741   0.96815   0.40669       314
           5    0.05116   0.96250   0.09716        80

    accuracy                        0.63159      9446
   macro avg    0.52282   0.67741   0.47750      9446
weighted avg    0.79003   0.63159   0.66991      9446



In [28]:
# test classification report
print(classification_report(y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.77419   0.29056   0.42254       413
           1    0.97706   0.92609   0.95089       460
           2    0.37500   0.31034   0.33962        58
           3    0.74862   0.61261   0.67382       666
           4    0.26131   0.98113   0.41270        53
           5    0.05634   0.94118   0.10631        17

    accuracy                        0.62388      1667
   macro avg    0.53209   0.67698   0.48431      1667
weighted avg    0.78244   0.62388   0.66231      1667



## Hyperparameter Tuning

In [29]:
param_grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=999)

For approach 1:

In [34]:
model_grid1 = RandomizedSearchCV(estimator = GaussianNB(), 
                            param_distributions = param_grid, 
                            verbose = 1, 
                            cv = cv_method,
                            n_iter = 100,
                            scoring = 'f1_weighted')
model_grid1.fit(X_train, y_train)
print('--------------------------------------------------------')
print('Best Estimator: ', model_grid1.best_estimator_)

Fitting 15 folds for each of 10 candidates, totalling 150 fits
--------------------------------------------------------
Best Estimator:  GaussianNB(var_smoothing=1.2328467394420635e-09)


In [35]:
# train classification report
y_train_pred = model_grid1.predict(X_train)
print(classification_report(y_train, y_train_pred, digits=5))

              precision    recall  f1-score   support

           0    0.87121   0.45423   0.59713      2338
           1    0.46764   0.97450   0.63200      2706
           2    0.22290   0.48344   0.30512       302
           3    0.85593   0.29978   0.44404      3706
           4    0.48284   0.67197   0.56192       314
           5    0.40404   1.00000   0.57554        80

    accuracy                        0.55547      9446
   macro avg    0.55076   0.64732   0.51929      9446
weighted avg    0.71201   0.55547   0.53637      9446



In [36]:
# test classification report
y_pred = model_grid1.predict(X_test)
print(classification_report(y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.85849   0.44068   0.58240       413
           1    0.46436   0.99130   0.63245       460
           2    0.20312   0.44828   0.27957        58
           3    0.88559   0.31381   0.46341       666
           4    0.44000   0.62264   0.51562        53
           5    0.47059   0.94118   0.62745        17

    accuracy                        0.55309      1667
   macro avg    0.55369   0.62631   0.51682      1667
weighted avg    0.72050   0.55309   0.53648      1667



For approach 2:

In [30]:
model_grid2 = RandomizedSearchCV(estimator = GaussianNB(), 
                            param_distributions = param_grid, 
                            verbose = 1, 
                            cv = cv_method,
                            n_iter = 100,
                            scoring = 'f1_weighted')
model_grid2.fit(X_new_train, y_train)
print('--------------------------------------------------------')
print('Best Estimator: ', model_grid2.best_estimator_)

Fitting 15 folds for each of 10 candidates, totalling 150 fits
--------------------------------------------------------
Best Estimator:  GaussianNB(var_smoothing=0.1873817422860384)


In [31]:
# train classification report
y_train_pred = model_grid2.predict(X_new_train)
print(classification_report(y_train, y_train_pred, digits=5))

              precision    recall  f1-score   support

           0    0.67773   0.45423   0.54392      2338
           1    0.94935   0.91426   0.93148      2706
           2    0.00000   0.00000   0.00000       302
           3    0.66471   0.94576   0.78071      3706
           4    0.00000   0.00000   0.00000       314
           5    0.00000   0.00000   0.00000        80

    accuracy                        0.74539      9446
   macro avg    0.38196   0.38571   0.37602      9446
weighted avg    0.70049   0.74539   0.70777      9446



In [32]:
# test classification report
y_pred = model_grid2.predict(X_new_test)
print(classification_report(y_test, y_pred, digits=5))

              precision    recall  f1-score   support

           0    0.68116   0.45521   0.54572       413
           1    0.96154   0.92391   0.94235       460
           2    0.00000   0.00000   0.00000        58
           3    0.65437   0.93243   0.76904       666
           4    0.00000   0.00000   0.00000        53
           5    0.00000   0.00000   0.00000        17

    accuracy                        0.74025      1667
   macro avg    0.38285   0.38526   0.37618      1667
weighted avg    0.69552   0.74025   0.70249      1667



## Best NB Model
Approach 2 has a higher recall score than approach 1.

## Run time

In [34]:
total_time = 0
for i in range(100):
    start_time = time()
    model_time = GaussianNB(var_smoothing=0.1873817422860384)
    clf_best = model_time.fit(X_new_train, y_train)
    y_pred = model_time.predict(X_new_test)
    run_time = time() - start_time
    total_time += run_time
avg_time = total_time / 100
print("Run time: ", avg_time)
print("Run time: ", run_time)

Run time:  0.11637187004089355


## Feature importance

In [35]:
imps = permutation_importance(model_time, X_test, y_test)

In [36]:
imps_sorted = imps.importances_mean.argsort()[::-1]

print(np.take(X_train.columns, imps_sorted[: 20]))

Index(['statuses_count', 'favourites_count', 'friends_count', 'tweets_49',
       'tweets_60', 'tweets_59', 'tweets_58', 'tweets_57', 'tweets_56',
       'tweets_55', 'tweets_54', 'tweets_53', 'tweets_52', 'tweets_51',
       'tweets_50', 'tweets_48', 'tweets_62', 'tweets_47', 'tweets_46',
       'tweets_45'],
      dtype='object')
